In this notebook, we will use [Lancet](https://ioam.github.io/lancet/) to perform a parameter sweep over 3 values of the ``rain_intensity`` parameter defined in [GSSHA_Workflow_Batched_Example2](GSSHA_Workflow_Batched_Example2.ipynb).

First import lancet:

In [ ]:
import lancet

## Declaring the ``Arguments``

Next we define our constant parameters using ``lancet.Args``:

In [ ]:
rain_duration = lancet.Args(rain_duration=3600)

Next we use ``lancet.List`` to define the three parameters we want to use for ``rain_intensity``:

In [ ]:
rain_intensity       = lancet.List('rain_intensity', [24,26,28])

Finally we take the cartesian product of these parameters using the ``*`` operator:

In [ ]:
args = rain_intensity * rain_duration

Here is the ``args`` object's repr:

In [ ]:
args

And the result of calling it's ``show`` method used to inspect the sets of arguments that will be executed:

In [ ]:
args.show()

## Defining ``ReportCommand``

Next we need to define how our arguments map to the ``param`` command described in [GSSHA_Workflow_Batched_Example2](GSSHA_Workflow_Batched_Example2.ipynb). To do this we define a ``ReportCommand`` subclass of ``lancet.Command``.

Note that this code is shown in this notebook to demonstrate how easy it is to interface lancet with an arbitrary command. Normally you would not show this code in a notebook and would simply import ``ReportCommand`` from the appropriate library.

In [ ]:
import param
import os
class ReportCommand(lancet.Command):
    
    notebook_path = param.String(doc='Path to the notebook used to generate the report')
    
    options = param.List(['--ExecutePreprocessor.allow_errors=True', 
                          '--ExecutePreprocessor.timeout=900'], doc="""
    Additional options to supply to nbconvert.""")

    def __init__(self, notebook_path, **params):
        super(ReportCommand,self).__init__(notebook_path=notebook_path,
                                          do_format=False,
                                          **params)
        self.pprint_args(['notebook_path'],['options'])
        
    def _fname(self, spec, tid, info):
        excluding = []
        root_dir = info['root_directory']
        params = [('tid' , tid)] + [(k,v) for  (k,v) in spec.items()
                                    if k in info['varying_keys']
                                    and k not in excluding]
        basename = '_'.join('%s=%s' % (k,v) for (k,v) in sorted(params))
        return os.path.join(root_dir, '%s_%s' % (info['batch_name'],
                                                   basename))
    def __call__(self, spec, tid=None, info={}):
        keywords = ['%s=%r' % (k,v) for k,v in spec.items()]
        params = []
        for kw in keywords:
            params.append('-p')
            params.append(kw)
            
        output_options = "--output-dir=%s --output %s" % (info['root_directory'], 
                                                          self._fname(spec, tid, info))
        inner_cmd = "jupyter nbconvert --execute %s %s %s" % (self.notebook_path, 
                                                              ' '.join(self.options),
                                                              output_options)
        return ['param', '-cmd', inner_cmd] + params
        

## Instantiating ``ReportCommand``

Now we instantiate ``ReportCommand`` by specifying the path to the notebook we want to use to generate reports. We then look at this objects ``repr``:

In [ ]:
notebook_path = os.path.abspath('GSSHA_Workflow_Batched_Example2.ipynb')
gssha_report = ReportCommand(notebook_path=notebook_path)
gssha_report

## Running the batch

Lastly, we declare a ``lancet.Launcher`` instance and call it to execute the batches locally. To run this on an Oracle Grid Engine cluster you should simply replace ``lancet.Launcher`` with ``lancet.QLauncher``.

A ``Launcher`` takes a name for the experimental run, the ``Args`` and ``Command`` objects and optional arguments such as the directory to collect results in (here it is ``'output'``) and a chosen limit on the number of concurrent process executed:

In [ ]:
lancet.Launcher('example', args, gssha_report, output_directory='output', max_concurrency=1)()

This command will take a few minutes to execute, blocking the notebook thread as the batches are being run locally. If ``QLauncher`` were used or another ``Launcher`` based on an HPC batch system, this would return as soon as the jobs are queued.

## Inspecting the results

The results are found in timestamped directories within the chosen ``output_directory`` which will be called ``output`` in this instance.

On Unix systems, you can see this by running the cell below:

In [ ]:
ls ./output

An example of the report output may be found in a location similar to ``output/2018-06-01_1353-example/example_rain_intensity=24_tid=0.html``. There will be as many such files and directories as batch jobs executed.